In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from datetime import datetime
from fgsm import  get_adversarial_exemple 

%load_ext tensorboard
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
    

2022-12-22 23:27:12.988740: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 23:27:14.190897: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-22 23:27:14.191044: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-22 23:27:14.191060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

time: 900 µs (started: 2022-12-22 23:27:16 +01:00)


/home/abdel/.local/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)


2022-12-22 23:27:16.428171: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-22 23:27:16.428242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-12-22 23:27:16.429075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


time: 221 ms (started: 2022-12-22 23:27:16 +01:00)


In [3]:
# convert tf.data.Dataset to numpy array
x_test = np.concatenate([x for x, y in ds_test], axis=0)
y_test = np.concatenate([y for x, y in ds_test], axis=0)
x_train = np.concatenate([x for x, y in ds_train], axis=0)
y_train = np.concatenate([y for x, y in ds_train], axis=0)

time: 5.26 s (started: 2022-12-22 23:27:16 +01:00)


In [4]:
class SequentialAdversarialLoss(tf.keras.Sequential):
    def __init__(self, *args, **kwargs):
        super(SequentialAdversarialLoss, self).__init__(*args, **kwargs)
        self.alpha = 0.5
        self.a = []
        self.eps = 0.25
    
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            #tape.watch(adv_x)
            y_pred = self(x, training=True)  # Forward pass
            
            #adv_y_pred = self(adv_x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred)
            
        gradient = tape.gradient(loss, x)
        signed_grad = tf.sign(gradient)
        x_adv = x + 0.25 * signed_grad
        x_adv = tf.clip_by_value(x_adv, 0, 1)
        
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x_adv)
            y_pred_adv = self(x_adv, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss *= self.alpha 
            loss += (1 - self.alpha) * self.compiled_loss(y, y_pred_adv)
            
            
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        del tape
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value

        return {m.name: m.result() for m in self.metrics}
    

time: 2.99 ms (started: 2022-12-22 23:27:21 +01:00)


In [5]:

maxout = SequentialAdversarialLoss([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(10, activation='softmax'),
])

maxout.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

maxout.fit(ds_train, epochs=20)

print("------------------Train dataset-------------")
maxout.evaluate(ds_train) 
x_train_adv = get_adversarial_exemple(maxout, ds_train, eps=0.25)
_, accuracy =  maxout.evaluate(x_train_adv, y_train, verbose = 0)
confidence = np.mean(np.max(maxout.predict(x_train_adv, verbose = 0), axis=1))
print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )
print("------------------Test dataset-------------")
maxout.evaluate(ds_test) 
x_test_adv = get_adversarial_exemple(maxout, ds_test, eps=0.25)
loss, accuracy =  maxout.evaluate(x_test_adv, y_test, verbose = 0)
confidence = np.mean(np.max(maxout.predict(x_test_adv, verbose = 0), axis=1))
print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )

Epoch 1/20
469/469 [==============================] - 14s 26ms/step - loss: 1.3239 - accuracy: 0.7324
Epoch 2/20
469/469 [==============================] - 12s 25ms/step - loss: 1.0056 - accuracy: 0.8506
Epoch 3/20
469/469 [==============================] - 12s 25ms/step - loss: 0.8989 - accuracy: 0.8763
Epoch 4/20
469/469 [==============================] - 12s 25ms/step - loss: 0.8334 - accuracy: 0.8895
Epoch 5/20
469/469 [==============================] - 12s 25ms/step - loss: 0.7829 - accuracy: 0.9005
Epoch 6/20
469/469 [==============================] - 12s 24ms/step - loss: 0.7525 - accuracy: 0.9038
Epoch 7/20
469/469 [==============================] - 12s 24ms/step - loss: 0.7296 - accuracy: 0.9089
Epoch 8/20
469/469 [==============================] - 12s 24ms/step - loss: 0.6976 - accuracy: 0.9154
Epoch 9/20
469/469 [==============================] - 12s 25ms/step - loss: 0.6892 - accuracy: 0.9166
Epoch 10/20
469/469 [==============================] - 12s 25ms/step - loss: 0.667

In [8]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

maxout = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(20, activation='softmax'),
])

maxout.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

maxout.fit(ds_train, epochs=20,callbacks=[tensorboard_callback])

print("------------------Train dataset-------------")
maxout.evaluate(ds_train) 
x_train_adv = get_adversarial_exemple(maxout, ds_train, eps=0.25)
_, accuracy =  maxout.evaluate(x_train_adv, y_train, verbose = 0)
print(f"Accuracy on adversarial examples: {accuracy}")
#confidence = np.mean(np.max(maxout.predict(x_train_adv, verbose = 0), axis=1))
#print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )

print("------------------Test dataset-------------")
maxout.evaluate(ds_test) 
x_test_adv = get_adversarial_exemple(maxout, ds_test, eps=0.25)
loss, accuracy =  maxout.evaluate(x_test_adv, y_test, verbose = 0)
print(f"Accuracy on adversarial examples: {accuracy}")
#confidence = np.mean(np.max(maxout.predict(x_test_adv, verbose = 0), axis=1))
#print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )

Epoch 1/20
469/469 [==============================] - 8s 15ms/step - loss: 0.4445 - accuracy: 0.8636
Epoch 2/20
469/469 [==============================] - 7s 15ms/step - loss: 0.1955 - accuracy: 0.9424
Epoch 3/20
469/469 [==============================] - 7s 15ms/step - loss: 0.1589 - accuracy: 0.9539
Epoch 4/20
469/469 [==============================] - 7s 15ms/step - loss: 0.1343 - accuracy: 0.9603
Epoch 5/20
469/469 [==============================] - 7s 15ms/step - loss: 0.1248 - accuracy: 0.9633
Epoch 6/20
469/469 [==============================] - 7s 15ms/step - loss: 0.1136 - accuracy: 0.9657
Epoch 7/20
469/469 [==============================] - 7s 15ms/step - loss: 0.1050 - accuracy: 0.9695
Epoch 8/20
469/469 [==============================] - 7s 15ms/step - loss: 0.1013 - accuracy: 0.9701
Epoch 9/20
469/469 [==============================] - 7s 15ms/step - loss: 0.0920 - accuracy: 0.9724
Epoch 10/20
469/469 [==============================] - 7s 15ms/step - loss: 0.0897 - accura

In [7]:
%tensorboard --logdir='logs/scalars'

time: 4.54 s (started: 2022-12-22 23:33:37 +01:00)
